In [ ]:
!pip install gigachain sentence_transformers faiss-gpu

In [4]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

## Embeddings

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from typing import List, Coroutine, Any


embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

## Get documents and slice them


In [22]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader

documents = TextLoader('test3.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"Total docs: {len(texts)}")

Total docs: 10


## Alternate Text splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[r'Глава [0-9]{1,}\.[\s\S]*?\n\n', r'Статья [0-9]{1,}\.[\s\S]*?\n\n'],
    is_separator_regex=True,
)
texts = text_splitter.split_documents(documents)

print(len(texts))

pretty_print_docs(texts)


## Model

In [27]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(
    credentials="",
    verify_ssl_certs=False
)

## CustomTextSplitter

In [37]:
import re

def split_text_with_regex(
    text: str, separator: str, keep_separator: bool
) -> List[str]:
    # Now that we have the separator, split the text
    if separator:
        if keep_separator:
            # The parentheses in the pattern keep the delimiters in the result.
            _splits = re.split(f"({separator})", text)
            splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
            if len(_splits) % 2 == 0:
                splits += _splits[-1:]
            splits = [_splits[0]] + splits
        else:
            splits = re.split(separator, text)
    else:
        splits = list(text)
    return [s for s in splits if s != ""]

## Summary pointers

In [40]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.base import Document

index_doc = 1
total_docs = []
for text in texts:
    # text_splitter = CharacterTextSplitter(chunk_overlap=0, separator=r'[0-9]{1,}\.\s', is_separator_regex=True)
    row_pointers = split_text_with_regex(text.page_content, r"[0-9]{1,}\.\s\w", keep_separator=True)

    pointers = text_splitter.create_documents(row_pointers)

    print(f"Total pointers: {len(pointers)}")

    chain = load_summarize_chain(llm, chain_type="refine")
    res = chain.run(pointers)

    if(res):
       res = f"44-ФЗ Глава{index_doc}\n{res}"

    index_doc += 1

    total_docs.append(Document(page_content=res))

pretty_print_docs(total_docs)

Total pointers: 2
Total pointers: 5
Total pointers: 9
Total pointers: 4
Total pointers: 17


KeyboardInterrupt: ignored

## Store documents

In [ ]:
from langchain.vectorstores import FAISS

retriever = FAISS.from_documents(texts, embeddings).as_retriever()

docs = retriever.get_relevant_documents("Какие основные аспекты регулируются федеральным законом о закупках для государственных и муниципальных нужд?")
pretty_print_docs(docs)

## Compress

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("Какие основные аспекты регулируются федеральным законом о закупках для государственных и муниципальных нужд?")
pretty_print_docs(compressed_docs)

## Get Answer

In [20]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """
Ответь на вопрос, только если в контексте есть информация, чтобы ответить на вопрос.
Если в контексте нет информации отвечай, что не знаешь ответа.
Используя только контекст и не свои знания ниже дай ответ на вопрос (Question). Не используй свои знания только контекст.
КОНТЕКСТ
-----
{context}
-----
Question: {question}

Если в контексте нет информации отвечай, что не знаешь ответа!
AI:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
)
for chunk in qa.stream("Какие основные аспекты регулируются федеральным законом о закупках для государственных и муниципальных нужд?"):
    print(chunk, end="", flush=True)

{'query': 'Какие основные аспекты регулируются федеральным законом о закупках для государственных и муниципальных нужд?', 'result': 'Основные аспекты, регулируемые федеральным законом о закупках для государственных и муниципальных нужд, включают:\n- определение поставщика (подрядчика, исполнителя) и заключение контракта;\n- закупка товара, работы, услуги для обеспечения государственных или муниципальных нужд;\n- права и обязанности заказчика, поставщика (подрядчика, исполнителя);\n- контроль за закупками и ответственность за нарушение законодательства о закупках.'}